In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='',
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38169?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Retail Associate, PT - Hyde Park Village

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresRetail Associate, PT - Hyde Park VillageTampa, FloridaBecome a Part of th

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res[0]

{'role': 'Retail Associate, PT - Hyde Park Village',
 'experience': 'Part Time –20-38 hours per week, including nights and weekend',
 'skills': ['Customer service',
  'Product knowledge',
  'Ability to learn and train on the latest products and technologies',
  'Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time'],
 'description': 'As a Nike Retail Associate, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way.'}

In [ ]:
file='/content/sample_data/my_portfolio.csv'
import pandas as pd

df = pd.read_csv(file)
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:07<00:00, 11.7MiB/s]


In [ ]:
type(json_res[0])

dict

In [ ]:
job = json_res[0]
job['skills']

['Customer service',
 'Product knowledge',
 'Ability to learn and train on the latest products and technologies',
 'Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time']

In [ ]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Varun Kumar, a business development executive at Vaku. Vaku is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Vaku
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Vaku portfolio: {link_list}
        Remember you are Mohan, BDE at Vaku.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Customer Experience and Operational Efficiency with Vaku's AI-Powered Solutions

Dear Hiring Manager,

I came across the job posting for a Retail Associate at Nike's Hyde Park Village location and was impressed by the emphasis on providing exceptional customer service and staying up-to-date with the latest products and technologies. As a Business Development Executive at Vaku, I believe our AI and software consulting expertise can help Nike streamline its retail operations, enhance customer experience, and drive business growth.

At Vaku, we specialize in developing tailored solutions that cater to the unique needs of our clients. Our team of experts has a proven track record of empowering enterprises with scalable, efficient, and cost-effective solutions. I'd like to highlight a few examples of our work that might be relevant to Nike's retail operations:

* Our Magento portfolio showcases our expertise in developing e-commerce solutions that provide seamless customer 